# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
#Load weather csv
#made dataframe 
weather = "weather_df.csv"
weather_df = pd.read_csv(weather)
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,hermiston,75,US,1587240622,30,45.84,-119.29,68.00,16.11
1,1,nishihara,1,JP,1587240622,94,26.18,127.76,71.60,5.82
2,2,ushuaia,75,AR,1587240622,69,-54.80,-68.30,50.00,18.34
3,3,kawalu,100,ID,1587240623,76,-7.38,108.21,72.61,1.68
4,4,inuvik,75,CA,1587240546,73,68.35,-133.72,19.40,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Gmaps configure
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
humidity_max = weather_df["Humidity"].max()

#Add Heatmap
fig = gmaps.figure(center=(25,0), zoom_level=1.75)
hlayer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=humidity_max)
fig.add_layer(hlayer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#Narrow Weather Conditions 
heat_df = weather_df.loc[(weather_df["Max Temp"] <80) & (weather_df["Wind Speed"]<2) &
                        (weather_df["Cloudiness"]<10)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = heat_df.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df["Hotel"] = ""
hotel_df

latitude = hotel_df["Lat"]
longitude = hotel_df["Lng"]
list_h = len(hotel_df)

In [16]:
for i in range(list_h):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude[i]},{longitude[i]}",
        "radius": 5000,
        "types": "lodging",
        "keyword": "Hotel",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    try: 
        results = response["results"][i]["name"]
        print(f"Nearest hotel: {results}")
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][i], "Hotel Name"] = results
    except IndexError:
        print("No options available")

Nearest hotel: Crossroads Hotel
No options available
Nearest hotel: Shengshi International Hotel
Nearest hotel: Sarnai restaurant&hotel
Nearest hotel: Hotel Camp Reinsehlen
Nearest hotel: Tokyu Harvest Club Katsuura
Nearest hotel: Hotel Antirrio
Nearest hotel: Hotel Apollon


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))